#### Using pandas to import the webpage

In [26]:
import pandas as pd
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

#### Check how many tables were imported

In [27]:
len(dfs)

3

#### Check what is in the first table

In [28]:
dfs[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Great this first table is what's needed!
#### Next assign the first table to a new df

In [29]:
df=dfs[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove rows with Boroughs that are marked as "Not Assigned"

In [30]:
df = df.set_index("Borough")
df = df.drop('Not assigned', axis=0)
df = df.reset_index()
df.head()

,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights


#### Reformat the df to the required order in terms of column positioning

In [31]:
df = df.set_index("Postcode")
df = df.reset_index()
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Check the number of rows in the df for use in the next step

In [32]:
df.shape

(211, 3)

#### Use While loop and if statements to:
##### 1. Assign Borough to Neighbourhood where Neighbourhood as marked as "Not Assigned"
##### 2. Combine the Neighbourhood for the same Postcode and drop additional rows with same Postcode

In [33]:
r=0
trow = 210
while r < trow:
    if df.iloc[r,2] == 'Not assigned':
        df.iloc[r,2] = df.iloc[r,1]
    if df.iloc[r,0] == df.iloc [r+1,0]:
        df.iloc[r,2] = df.iloc[r,2] + ", " + df.iloc[r+1,2]
        df = df.drop([r+1], axis=0)
        df = df.reset_index(drop=True)
        trow = trow - 1
        r = r - 1
    r = r + 1
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## Answer to Part 1, print the shape of the processed df

In [34]:
df.shape

(103, 3)

### Part 2 of the Assignment
#### Combining the latitude and longtitude data

In [35]:
!wget -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data

--2019-06-15 09:03:05--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-06-15 09:03:05--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-15 09:03:09--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-15 09:03:09--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [36]:
Geodata = pd.read_csv("Geospatial_data.csv", delimiter=",")
Geodata[0:7]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029


In [37]:
len(list(df['Postcode'].unique()))

103

In [38]:
len(list(Geodata['Postal Code'].unique()))

103

In [39]:
result = pd.merge(df,
                  Geodata,
                  left_on='Postcode',right_on='Postal Code')
result.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


In [40]:
result=result.drop(columns=['Postal Code'])

## ANSWER TO PART 2

In [41]:
result

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [42]:
result.to_csv('Toronto.csv',index=False)

In [43]:
df = pd.read_csv("Toronto.csv", delimiter=",")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [44]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [45]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [46]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [47]:
# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Postcode']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [48]:
to_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
to_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [49]:
# create map of Toronto using latitude and longitude values
map_toto = folium.Map(location=[latitude, longitude], zoom_start=10)
dfbu=df
df = to_data

# add markers to map
for lat, lng, borough, postcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Postcode']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toto)  
    
map_toto

In [50]:
toronto_clustering = to_data.drop(columns=['Postcode','Borough','Neighbourhood'])
toronto_clustering.head()

,Latitude,Longitude
0,43.654260,-79.360636
1,43.657162,-79.378937
2,43.651494,-79.375418
3,43.676357,-79.293031
4,43.644771,-79.373306


In [51]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

## Clustering based on the latitude and longtitude of the postcodes

In [52]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 0, 2, 0, 2, 0], dtype=int32)

In [53]:
# add clustering labels
to_clusters = to_data
to_clusters.insert(0, 'Cluster Labels', kmeans.labels_)
to_clusters

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,0,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,2,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,2,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,0,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [54]:
import numpy as np # library to handle data in a vectorized manner

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_clusters['Latitude'], to_clusters['Longitude'], to_clusters['Postcode'], to_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters